<a href="https://colab.research.google.com/github/Lubraca/export-credit-risk-lab/blob/main/Notebooks/sce_indenization_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Configuring git

!git config --global user.name "Lubraca"
!git config --global user.email "lucas.casarin@hotmail.com"

In [ ]:
# clone repository

!git clone https://github.com/Lubraca/export-credit-risk-lab

Cloning into 'export-credit-risk-lab'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 444.22 KiB | 3.80 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
%cd export-credit-risk-lab

/content/export-credit-risk-lab


Python Packages

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 9.4 MB/s eta 0:00:00


In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import unidecode

In [ ]:
# import data from indenizations

# Dataset with insurance compensation for Brazilian Export Credit Insurance issued from ABGF available in the link below (there is a spreadsheet link in the bottom of the page):
# https://www.gov.br/mdic/pt-br/assuntos/camex/se-camex/sucex/financiamento/sce/indenizacoes-do-seguro-de-credito-a-exportacao

ic_df = pd.read_csv('/content/export-credit-risk-lab/pagamento_de_indenizacoes_Janeiro2026.csv', encoding='latin1')

# import data from BNDES

post_ship = pd.read_csv('/content/export-credit-risk-lab/operacoes-exportacao-operacoes-de-exportacao-pos-embarque-bens.csv', encoding='latin1', sep=';')

eng_serv = pd.read_csv('/content/export-credit-risk-lab/operacoes-exportacao-operacoes-de-exportacao-pos-embarque-servicos-de-engenharia.csv', encoding='latin1', sep=';')

pre_ship = pd.read_csv('/content/export-credit-risk-lab/operacoes-exportacao-operacoes-de-exportacao-pre-embarque.csv', encoding='latin1', sep=';')

This exploration

Data Preparation

### Helper Functions for Data Preparation and Transformation

This cell defines a collection of helper functions designed to clean, standardize, and transform the imported dataframes. These functions include:

- `standardize`: Standardizes column names and converts text columns to lowercase.
- `apply_corrections`: Applies custom corrections to specified columns using a mapping.
- `normalize_name`: Normalizes names by removing special characters, common legal suffixes (like S/A, Ltda), and sorting words.
- `add_norm_names`: Adds normalized name columns for 'exportador', 'importador', and 'beneficiario'.
- `parse_dates`: Converts specified columns to datetime objects.
- `add_contract_year`: Extracts the contract year from a 'certificado de garantia de cobertura' column.
- `convert_value`: Converts monetary values from string to float, handling 'USD' currency.
- `normalize_keyword`: Normalizes keywords for categorization.
- `add_object_category`: Categorizes export objects based on keywords.
- `set_categoricals`: Converts specified columns to categorical data types.
- `run_etl`: Orchestrates the execution of the above helper functions to perform a complete ETL (Extract, Transform, Load) process on a dataframe.

In [88]:
from typing import Dict, Optional, Callable

# Remove special characters from df column names and set them lowercase

def standardize(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [unidecode.unidecode(str(c)).lower().strip() for c in df.columns]
    text_cols = df.select_dtypes(include=["object"]).columns
    for c in text_cols:
        df[c] = df[c].astype("string").str.lower().str.strip()
    return df

# support function that removes any diacritic mark in every object that it is applied

def _norm_map_val(x: object) -> object:
    if x is None or pd.isna(x):
        return pd.NA
    return unidecode.unidecode(str(x)).lower().strip()

# Function that maps texts before and after normalization with the function above "_norm_map_val"

def _norm_mapping(mapping: Dict[str, str]) -> Dict[str, str]:
    out: Dict[str, str] = {}
    for k, v in mapping.items():
        nk, nv = _norm_map_val(k), _norm_map_val(v)
        if pd.isna(nk) or pd.isna(nv):
            continue
        out[str(nk)] = str(nv)
    return out

# Check the columns value and analyse if them were treat my "_norm_mapping", if True is returns the normalized value
# otherwise, it keeps the value as it is

def apply_corrections(df: pd.DataFrame, corrections_by_column: Dict[str, Dict[str, str]]) -> pd.DataFrame:
    df = df.copy()
    for col, mapping in corrections_by_column.items():
        if col not in df.columns:
            continue
        df[col] = df[col].astype("string")
        mp = _norm_mapping(mapping)

        def _fix(x: object) -> object:
            nx = _norm_map_val(x)
            if pd.isna(nx):
                return pd.NA
            return mp.get(str(nx), x)

        df[col] = df[col].map(_fix)
    return df

# normalization of exporters, importers and beneficiary name

def normalize_name(x: object) -> object:
    if x is None or pd.isna(x):
        return pd.NA
    s = unidecode.unidecode(str(x)).lower().strip()
# uses regular expressions to remove common Brazilian legal entity suffixes
    s = re.sub(r"\b(sa|s/a|ltda|epp|me)\b", "", s)
# remove any character that is not a lowercase letter (a-z) or a space
    s = re.sub(r"[^a-z ]", "", s)
# reoder the words in alphabetical order
    s = " ".join(sorted(s.split()))
# remove leading and trailing spaces
    s = s.strip()
    return s if s else pd.NA

# Apply the function "normalize_name" for importer, exporter and beneficiary values

def add_norm_names(df: pd.DataFrame, cols: Optional[list[str]] = None) -> pd.DataFrame:
    df = df.copy()
    cols = cols or ["exportador", "importador", "beneficiario", 'pais',
       'objeto da exportacao', 'financiador', 'modalidade de financiamento',
       'natureza do risco', 'forma de pagamento do premio' ]
    for c in cols:
        if c in df.columns:
            df[f"{c}_norm"] = df[c].astype("string").map(normalize_name)
    return df

# converts specified columns to datetime objects, made more robust
def parse_dates(df: pd.DataFrame, cols: Optional[list[str]] = None) -> pd.DataFrame:
    df = df.copy()

    # Define a list of all potential date columns across various dataframes
    potential_date_cols = [
        "autorizacao da indenizacao",
        "data da inadimplencia",
        "data da contratação", # Found in some datasets, e.g., pre-ship before standardize
        "data_da_contratacao", # Found in BNDES datasets (post_ship, eng_serv) after standardize
        "data de aprovacao da operacao" # Found in ic_df
    ]

    # If `cols` is provided, use it. Otherwise, use the potential_date_cols.
    columns_to_process = cols if cols is not None else potential_date_cols

    for col in columns_to_process:
        if col in df.columns: # Crucially, only process if the column exists in the DataFrame
            if col == "autorizacao da indenizacao":
                df[col] = pd.to_datetime(df[col], format="%d-%m-%Y", errors="coerce")
            elif col == "data da inadimplencia":
                df[col] = pd.to_datetime(df[col], errors="coerce")
                df["ano de inadimplencia"] = df[col].dt.year
            elif col == "data da contratação" or col == "data_da_contratacao":
                df[col] = pd.to_datetime(df[col], errors="coerce")
                df[f"ano_da_{col.replace('da_', '')}"] = df[col].dt.year # Create year column for contract dates
            elif col == "data de aprovacao da operacao":
                df[col] = pd.to_datetime(df[col], errors="coerce")
                df["ano_da_aprovacao_da_operacao"] = df[col].dt.year # Create year column for approval dates
            else:
                # Generic handling for any other date column not explicitly listed above
                df[col] = pd.to_datetime(df[col], errors="coerce")
                if 'ano' not in col: # Avoid creating redundant year columns
                    df[f"ano_{col.replace(' ', '_')}"] = df[col].dt.year

    return df

# Extract the starting date for insurance covarage

def add_contract_year(df: pd.DataFrame, contract_years_regex: str) -> pd.DataFrame:
    df = df.copy()
    col = "certificado de garantia de cobertura"
    if col in df.columns:
        df["ano do contrato"] = df[col].astype("string").str.extract(contract_years_regex).astype("float64")
    return df

# indenizations cost converted from string and USD symbol to float

def convert_usd_column(df: pd.DataFrame, col: str = "valor total da indenizacao") -> pd.DataFrame:
    df = df.copy()
    if col in df.columns:
        def _remove_and_convert_usd(valor):
            if isinstance(valor, str) and "USD" in valor:
                valor = valor.replace("USD", "").replace(".", "").replace(",", ".").strip()
                return float(valor)
            return valor
        df[col] = df[col].apply(_remove_and_convert_usd)
    return df

# normalizes keywords for categorization.

def normalize_keyword(x: object) -> object:
    if x is None or pd.isna(x):
        return pd.NA
    s = unidecode.unidecode(str(x)).lower().strip()
    s = re.sub(r"[^a-z0-9 ]", "", s)
    return s if s else pd.NA

# categorizes export objects based on keywords

def add_object_category(df: pd.DataFrame, categorize: Dict[str, list[str]],
                        source_col: str = "objeto da exportacao",
                        target_col: str = "objeto_categoria") -> pd.DataFrame:
    df = df.copy()
    if source_col not in df.columns:
        return df

    cat = {k: [str(normalize_keyword(v)) for v in vs] for k, vs in categorize.items()}

    def _classify(x: object) -> object:
        s = normalize_keyword(x)
        if pd.isna(s):
            return pd.NA
        s = str(s)
        for k, kws in cat.items():
            if any(kw in s for kw in kws):
                return k
        return "outros"

    df[target_col] = df[source_col].map(_classify)
    return df

# converts specified columns to categorical data types

def set_categoricals(df: pd.DataFrame, cols: Optional[list[str]] = None) -> pd.DataFrame:
    df = df.copy()
    cols = cols or [
        "exportador", "importador", "beneficiario",
        "exportador_norm", "importador_norm", "beneficiario_norm",
        "pais", "objeto da exportacao", "financiador",
        "modalidade de financiamento", "natureza do risco",
        "forma de pagamento do premio",
        "objeto_categoria",
    ]
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype("category")
    return df


def run_etl_sce(
    df: pd.DataFrame,
    contract_years_regex: str,
    categorize: Optional[Dict[str, list[str]]] = None
) -> pd.DataFrame:
    df2 = standardize(df)
    df2 = add_norm_names(df2)
    df2 = parse_dates(df2) # This will now safely attempt to parse known SCE date columns
    df2 = add_contract_year(df2, contract_years_regex)
    df2 = convert_usd_column(df2)
    if categorize:
        df2 = add_object_category(df2, categorize)
    df2 = set_categoricals(df2)
    return df2


In [55]:
# Create year time variable from the contract order (assumimng that the year present in the order number represents the contrant year)

years = np.arange(start = 2000, stop = 2026)
contract_years = "|".join(map(str,years))

categorize = {
    'Aeronaves': ['Aeronaves','aeronave', 'Aerovane' 'aeronaves', 'EMB', 'Avião', 'avião', 'Boeing', 'Airbus', 'Embraer', 'Helicóptero', 'helicoptero', 'Helicópteros', 'helicopteros'],
    'Construção Civil': ['Elementos','Estrutura','construção', 'obras', 'aeroporto', 'porto', 'infraestrutura', 'metrô','edificações'],
    'Agronegócio': ['agrícola', 'colheitadeiras', 'tratores', 'arroz', 'fertilizantes', 'plantio', 'proteína','cana'],
    'Equipamentos': ['Equpamentos','Equipamentos','Sistema','Máquina', 'Máquinas','maquina','máquinas', 'equipamentos', 'Prensas', 'prensas', 'sistemas', 'hidráulicos','Sonda','Extrator', 'Bens'],
    'Produtos': ['Produtos','produtos','químicos','prensas','Lâminas', 'lâminas', 'cremes', 'pinturas', 'Impressora','produção','Móveis', 'Módulos', 'Elevadores','Balanças'],
    'Veículos': ['Colheitadeiras','Escavadeiras','Caminhões','caminhões', 'veículos', 'retroescavadeiras', 'escavadeiras', "caminhão", "ônibus", "carros", "veículo"],
}

contract_years_regex=r"\b(200\d|201\d|202[0-5])\b"

In [89]:
df_sce_gold = run_etl_sce(
    df=ic_df,
    contract_years_regex=contract_years_regex,
    categorize=categorize,   # optional, remove if not needed

)

/tmp/ipython-input-555074359.py:106: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


In [57]:
df_sce_gold.columns

Index(['certificado de garantia de cobertura', 'data da inadimplencia',
       'autorizacao da indenizacao', 'valor total da indenizacao',
       'exportador', 'importador', 'beneficiario', 'pais',
       'objeto da exportacao', 'financiador', 'modalidade de financiamento',
       'natureza do risco', 'forma de pagamento do premio',
       'data de aprovacao da operacao', 'instancia responsavel pela aprovacao',
       'termo de quitacao', 'titulo de credito',
       'termo de quitacao sei (valor da indenizacao)',
       'titulos de credito sei (valor da parcela)', 'verificacao sei (nome)',
       'observacao', 'exportador_norm', 'importador_norm', 'beneficiario_norm',
       'pais_norm', 'objeto da exportacao_norm', 'financiador_norm',
       'modalidade de financiamento_norm', 'natureza do risco_norm',
       'forma de pagamento do premio_norm', 'ano de inadimplencia',
       'ano do contrato', 'objeto_categoria'],
      dtype='object')

In [58]:
df_sce_gold.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106 entries, 0 to 1105
Data columns (total 33 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   certificado de garantia de cobertura          1106 non-null   string        
 1   data da inadimplencia                         208 non-null    datetime64[ns]
 2   autorizacao da indenizacao                    0 non-null      datetime64[ns]
 3   valor total da indenizacao                    1106 non-null   object        
 4   exportador                                    1106 non-null   category      
 5   importador                                    1106 non-null   category      
 6   beneficiario                                  1106 non-null   category      
 7   pais                                          1106 non-null   category      
 8   objeto da exportacao                          1106 non-null   catego

In [61]:
df_sce_gold['modalidade de financiamento_norm'].unique()

array(['ao buyers credit financiamento importador',
       'com do exportador posembarque proprios recursos',
       'ao credit exportador financiamento suppliers',
       'ao credit exportador financiamento supplierss', 'credit supplier',
       'buyers credit'], dtype=object)

In [59]:
df_sce_gold[["exportador", "exportador_norm"]].head()

,exportador,exportador_norm
0,construtora norberto odebrecht s.a.,construtora norberto odebrecht sa
1,construtora norberto odebrecht s.a.,construtora norberto odebrecht sa
2,construtora norberto odebrecht s.a.,construtora norberto odebrecht sa
3,construtora norberto odebrecht s.a.,construtora norberto odebrecht sa
4,alpha química ltda.,alpha quimica


In [ ]:
df_sce_gold["objeto_categoria"].value_counts(dropna=False)


,count
objeto_categoria,
Agronegócio,341
Construção Civil,313
Equipamentos,244
Aeronaves,117
Veículos,45
Produtos,35
outros,11


In [ ]:
df_sce_gold["ano do contrato"].value_counts(dropna=False).sort_index()

,count
ano do contrato,
2008.0,161
2009.0,76
2010.0,352
2011.0,92
2012.0,143
2013.0,49
2014.0,118
2015.0,32
2016.0,40


In [62]:
df_sce_gold.to_csv('df_sce_gold.csv', index=False)
print("df_sce_gold.csv saved successfully!")

df_sce_gold.csv saved successfully!


Data preparation for financing data

In [ ]:
print( f'Post Shippiment base shape: {post_ship.shape}')

print(f'Pre Shippiment base shape: {pre_ship.shape}')

print(f'Eng Serv base shape: {eng_serv.shape}')

Post Shippiment base shape: (2174, 21)
Pre Shippiment base shape: (7587, 26)
Eng Serv base shape: (652, 25)


Analize the coluns that will be used as keys for joining df_sce_gold with each indenization table

In [ ]:

post_ship.columns

Index(['exportador', 'cnpj_do_exportador', 'porte_do_exportador',
       'descricao_da_operacao', 'uf', 'pais_destino_das_exportacoes',
       'numero_da_operacao', 'data_da_contratacao', 'moeda_sigla',
       'fonte_de_recursos_desembolsos', 'custo_financeiro',
       'setor_subsetor_de_atividade', 'area_operacional',
       'modalidade_de_apoio', 'forma_de_apoio', 'produto',
       'modalidade_operacional', 'mutuario', 'categoria',
       'situacao_da_operacao', 'tipo_de_garantia'],
      dtype='object')

In [65]:
post_ship[['exportador', 'pais_destino_das_exportacoes', 'setor_subsetor_de_atividade', 'data_da_contratacao']].head(10)

,exportador,pais_destino_das_exportacoes,setor_subsetor_de_atividade,data_da_contratacao
0,MARCOPOLO SA,BOLIVIA,"INDUSTRIA/FABRICACAO DE VEICULOS AUTOMOTORES, ...",2002-01-04
1,MAQUINAS AGRICOLAS JACTO S A,ARGENTINA,INDUSTRIA/FABRICACAO DE MAQUINAS E EQUIPAMENTOS,2002-01-08
2,MAQUINAS AGRICOLAS JACTO S A,ARGENTINA,INDUSTRIA/FABRICACAO DE MAQUINAS E EQUIPAMENTOS,2002-01-08
3,SCHULZ S/A,ESTADOS UNIDOS,INDUSTRIA/METALURGIA,2002-01-08
4,VOLVO DO BRASIL VEICULOS LTDA,ARGENTINA,"INDUSTRIA/FABRICACAO DE VEICULOS AUTOMOTORES, ...",2002-01-08
5,PERFIPAR S/A - MANUFATURADOS DE ACO,ARGENTINA,INDUSTRIA/METALURGIA,2002-01-09
6,PERFIPAR S/A - MANUFATURADOS DE ACO,ARGENTINA,INDUSTRIA/METALURGIA,2002-01-09
7,PERFIPAR S/A - MANUFATURADOS DE ACO,ARGENTINA,INDUSTRIA/METALURGIA,2002-01-09
8,PERFIPAR S/A - MANUFATURADOS DE ACO,ARGENTINA,INDUSTRIA/METALURGIA,2002-01-09
9,PERFIPAR S/A - MANUFATURADOS DE ACO,ARGENTINA,INDUSTRIA/METALURGIA,2002-01-09


In [ ]:
pre_ship.columns

Index(['cliente', 'cpf_cnpj', 'uf', 'municipio', 'municipio_codigo',
       'data_da_contratacao', 'valor_da_operacao_em_reais',
       'valor_desembolsado_em_reais', 'fonte_de_recurso_desembolsos',
       'modalidade_de_apoio', 'forma_de_apoio', 'produto',
       'instrumento_financeiro', 'inovacao', 'area_operacional', 'setor_cnae',
       'subsetor_cnae_agrupado', 'subsetor_cnae_codigo', 'subsetor_cnae_nome',
       'setor_bndes', 'subsetor_bndes', 'porte_do_cliente',
       'natureza_do_cliente', 'instituicao_financeira_credenciada',
       'cnpj_do_agente_financeiro', 'situacao_da_operacao'],
      dtype='object')

In [72]:
pre_ship[['cliente','data_da_contratacao','subsetor_bndes']].head(10)

,cliente,data_da_contratacao,subsetor_bndes
0,MINERVA S.A.,08/01/2002,ALIMENTO E BEBIDA
1,MINERVA S.A.,08/01/2002,ALIMENTO E BEBIDA
2,FRUTIVITA S/A,09/01/2002,AGROPECUÁRIA
3,CALCADOS CHICARONI LTDA,10/01/2002,OUTRAS
4,MINERVA S.A.,10/01/2002,ALIMENTO E BEBIDA
5,SANRISIL S/A INDUSTRIA E COMERCIO,14/01/2002,QUÍMICA E PETROQUÍMICA
6,KEPLER WEBER SA,14/01/2002,MECÂNICA
7,KEPLER WEBER SA,14/01/2002,MECÂNICA
8,KEPLER WEBER SA,14/01/2002,MECÂNICA
9,TUPY S/A,17/01/2002,MATERIAL DE TRANSPORTE


In [ ]:
eng_serv.columns

Index(['exportador', 'cnpj_do_exportador', 'porte_do_exportador',
       'descricao_da_operacao', 'uf', 'pais_destino_das_exportacoes',
       'numero_da_operacao', 'data_da_contratacao', 'valor_da_operacao_em_um',
       'valor_desembolsado_em_um', 'moeda_sigla',
       'fonte_de_recursos_desembolsos', 'custo_financeiro', 'juros',
       'prazo_total_meses', 'setor_subsetor_de_atividade', 'area_operacional',
       'modalidade_de_apoio', 'forma_de_apoio', 'produto',
       'modalidade_operacional', 'mutuario', 'categoria',
       'situacao_da_operacao', 'tipo_de_garantia'],
      dtype='object')

In [73]:
eng_serv[['exportador', 'pais_destino_das_exportacoes',
       'data_da_contratacao','setor_subsetor_de_atividade']].head(10)

,exportador,pais_destino_das_exportacoes,data_da_contratacao,setor_subsetor_de_atividade
0,MULTITRADE S/A,EQUADOR,1998-07-24,COMERCIO E SERVICOS/CONSTRUÇÃO
1,MULTITRADE S/A,EQUADOR,1998-07-24,COMERCIO E SERVICOS/CONSTRUÇÃO
2,CONSTRUTORA ANDRADE GUTIERREZ S/A,EQUADOR,1998-08-07,COMERCIO E SERVICOS/CONSTRUÇÃO
3,MULTITRADE S/A,EQUADOR,1998-08-07,COMERCIO E SERVICOS/CONSTRUÇÃO
4,MULTITRADE S/A,EQUADOR,1998-09-17,COMERCIO E SERVICOS/CONSTRUÇÃO
5,MULTITRADE S/A,EQUADOR,1998-11-17,COMERCIO E SERVICOS/CONSTRUÇÃO
6,MULTITRADE S/A,EQUADOR,1998-11-20,COMERCIO E SERVICOS/CONSTRUÇÃO
7,MULTITRADE S/A,EQUADOR,1998-11-26,COMERCIO E SERVICOS/CONSTRUÇÃO
8,MULTITRADE S/A,EQUADOR,1998-12-16,COMERCIO E SERVICOS/CONSTRUÇÃO
9,CONSTRUTORA NORBERTO ODEBRECHT SA,EQUADOR,1999-02-10,COMERCIO E SERVICOS/CONSTRUÇÃO


In [74]:
# After an analization, the columns that fit as surrogate keys (strong column to join two table based with the concatenation of two/+ columns)
post_ship_sur_k = ['exportador', 'pais_destino_das_exportacoes', 'setor_subsetor_de_atividade', 'data_da_contratacao']
pre_ship_sur_key = ['cliente','data_da_contratacao','subsetor_bndes']
eng_serv_sur_key = ['exportador', 'pais_destino_das_exportacoes','data_da_contratacao','setor_subsetor_de_atividade']

In [79]:
def run_etl_bndes(
    df: pd.DataFrame) -> pd.DataFrame:
    df2 = standardize(df)
    df2 = add_norm_names(df2),
    df2 = parse_dates(df2),
    return df2

In [85]:
run_etl_bndes(post_ship)

,exportador,cnpj_do_exportador,porte_do_exportador,descricao_da_operacao,uf,pais_destino_das_exportacoes,numero_da_operacao,data_da_contratacao,moeda_sigla,fonte_de_recursos_desembolsos,...,modalidade_de_apoio,forma_de_apoio,produto,modalidade_operacional,mutuario,categoria,situacao_da_operacao,tipo_de_garantia,exportador_norm,ano_da_data_contratacao
0,marcopolo sa,88.611.835/0001-29,grande,exportação de bens,rs,bolivia,0865484,2002-01-04,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque,supplier,ente privado,bens de capital e demais operações,liquidada,seguro de crédito/fge / pessoal,marcopolo,2002
1,maquinas agricolas jacto s a,55.064.562/0001-90,grande,exportação de bens,sp,argentina,0938049,2002-01-08,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque,supplier,ente privado,bens de capital e demais operações,liquidada,pessoal,a agricolas jacto maquinas s,2002
2,maquinas agricolas jacto s a,55.064.562/0001-90,grande,exportação de bens,sp,argentina,0938050,2002-01-08,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque,supplier,ente privado,bens de capital e demais operações,liquidada,pessoal,a agricolas jacto maquinas s,2002
3,schulz s/a,84.693.183/0001-68,grande,exportação de bens,sc,estados unidos,0704109,2002-01-08,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque,supplier,ente privado,bens de capital e demais operações,liquidada,pessoal,schulz,2002
4,volvo do brasil veiculos ltda,43.999.424/0001-14,grande,exportação de bens,pr,argentina,0894220,2002-01-08,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque,supplier,ente privado,bens de capital e demais operações,liquidada,seguro de crédito/fge / pessoal,brasil do veiculos volvo,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,kepler weber industrial s/a,87.288.940/0001-06,grande,exportação de bens,rs,paraguai,7514985/0001,2025-11-12,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque - exim automático,supplier,ente privado,bens de capital e demais operações,ativa,pessoal,industrial kepler weber,2025
2170,kepler weber industrial s/a,87.288.940/0001-06,grande,exportação de bens,rs,paraguai,7511369/0002,2025-11-18,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque - exim automático,supplier,ente privado,bens de capital e demais operações,ativa,pessoal,industrial kepler weber,2025
2171,nb maquinas ltda,46.127.635/0001-55,grande,exportação de bens,sp,equador,7522475/0001,2025-11-18,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque - exim automático,supplier,ente privado,bens de capital e demais operações,ativa,pessoal,maquinas nb,2025
2172,hece maquinas ltda,59.602.144/0001-60,média,exportação de bens,sp,equador,7538038/0001,2025-11-25,us$ compra,fat cambial,...,reembolsável,direta,bndes exim pós-embarque - exim automático,supplier,ente privado,bens de capital e demais operações,ativa,pessoal,hece maquinas,2025


In [86]:
run_etl_bndes(pre_ship)

,cliente,cpf_cnpj,uf,municipio,municipio_codigo,data_da_contratacao,valor_da_operacao_em_reais,valor_desembolsado_em_reais,fonte_de_recurso_desembolsos,modalidade_de_apoio,...,subsetor_cnae_codigo,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_do_agente_financeiro,situacao_da_operacao,ano_da_data_contratacao
0,minerva s.a.,**.*20.377/0001-**,sp,barretos,3505500,2002-08-01,7.028.400,7.035.625,recursos livres - fat,reembolsável,...,c1011200,"abate de reses, exceto suinos",industria,alimento e bebida,grande,privada,boston negocios e participacoes ltda.,33.140.666.0001-02,liquidada,2002.0
1,minerva s.a.,**.*20.377/0001-**,sp,barretos,3505500,2002-08-01,7.028.400,7.035.625,recursos livres - fat,reembolsável,...,c1011200,"abate de reses, exceto suinos",industria,alimento e bebida,grande,privada,boston negocios e participacoes ltda.,33.140.666.0001-02,liquidada,2002.0
2,frutivita s/a,**.*09.838/0001-**,pe,petrolina,2611101,2002-09-01,485.967,486.416,recursos livres - fat,reembolsável,...,a0119900,cult plantas lavoura temporaria nao especifica...,agropecuária,agropecuária,pequena,privada,banco bradesco financiamentos s.a.,07.207.996.0001-50,liquidada,2002.0
3,calcados chicaroni ltda,**.*51.979/0001-**,sp,franca,3516200,2002-10-01,120.469,120.618,recursos livres - fat,reembolsável,...,c1530000,fabricacao de calcados,industria,outras,média,privada,banco fibra sa,58.616.418.0001-08,liquidada,2002.0
4,minerva s.a.,**.*20.377/0001-**,sp,barretos,3505500,2002-10-01,4.758.800,4.765.159,recursos livres - fat,reembolsável,...,c1011200,"abate de reses, exceto suinos",industria,alimento e bebida,grande,privada,itau unibanco s.a.,60.701.190.0001-04,liquidada,2002.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7582,veloso green coffee exportacao ltda,**.*49.347/0001-**,mg,carmo do paranaiba,3114303,2025-10-12,6.000.000,6.000.000,-,reembolsável,...,g4621400,comercio atacadista de cafe em grao,comercio/servicos,comércio e serviços,grande,privada,banco safra s a,58.160.789.0001-28,ativa,2025.0
7583,companhia brasileira de cartuchos,**.*94.031/0001-**,sp,ribeirao pires,3543303,2025-12-12,24.949.020,25.639.940,recursos livres - fat cambial,reembolsável,...,c2550102,"fabricacao de armas de fogo, outras armas e mu...",industria,metalurgia e produtos,grande,privada,banco safra s a,58.160.789.0001-28,ativa,2025.0
7584,crystal sshoes u assessoria e lancamentos ltda,**.*62.853/0001-**,rs,novo hamburgo,4313409,2025-12-12,2.494.902,<NA>,-,reembolsável,...,c1531901,fabricacao de calcados de couro,industria,outras,média,privada,itau unibanco s.a.,60.701.190.0001-04,ativa,2025.0
7585,marcopolo sa,**.*11.835/0001-**,rs,caxias do sul,4305108,NaT,496.440.000,498.717.000,recursos livres - fat cambial,reembolsável,...,c2920401,fabricacao de caminhoes e onibus,industria,material de transporte,grande,privada,sem nome,00.000.000.0000-00,ativa,NaN


In [87]:
run_etl_bndes(eng_serv)

,exportador,cnpj_do_exportador,porte_do_exportador,descricao_da_operacao,uf,pais_destino_das_exportacoes,numero_da_operacao,data_da_contratacao,valor_da_operacao_em_um,valor_desembolsado_em_um,...,modalidade_de_apoio,forma_de_apoio,produto,modalidade_operacional,mutuario,categoria,situacao_da_operacao,tipo_de_garantia,exportador_norm,ano_da_data_contratacao
0,multitrade s/a,15.144.298/0001-80,grande,exportação de bens e serviços para construção ...,rj,equador,551515,1998-07-24,"5230058,91","5230058,91",...,reembolsável,direta,bndes exim pós-embarque,supplier,ente público,bens e serviços para obras no exterior,liquidada,ccr/ aladi,multitrade,1998
1,multitrade s/a,15.144.298/0001-80,grande,exportação de bens e serviços para construção ...,rj,equador,551515,1998-07-24,"8092188,49","8092188,49",...,reembolsável,direta,bndes exim pós-embarque,supplier,ente público,bens e serviços para obras no exterior,liquidada,ccr/ aladi,multitrade,1998
2,construtora andrade gutierrez s/a,17.262.213/0001-94,grande,exportacao de bens e servicos para irrigacao t...,mg,equador,572701,1998-08-07,"64400000,0","63847527,21999999",...,reembolsável,direta,bndes exim pós-embarque,buyer,ente público,bens e serviços para obras no exterior,liquidada,ccr/ aladi,andrade construtora gutierrez,1998
3,multitrade s/a,15.144.298/0001-80,grande,exportação de bens e serviços para construção ...,rj,equador,551515,1998-08-07,"1445847,08","1445847,08",...,reembolsável,direta,bndes exim pós-embarque,supplier,ente público,bens e serviços para obras no exterior,liquidada,ccr/ aladi,multitrade,1998
4,multitrade s/a,15.144.298/0001-80,grande,exportação de bens e serviços para construção ...,rj,equador,551515,1998-09-17,"3065500,05","3065500,05",...,reembolsável,direta,bndes exim pós-embarque,supplier,ente público,bens e serviços para obras no exterior,liquidada,ccr/ aladi,multitrade,1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,construtora norberto odebrecht sa,15.102.288/0001-82,grande,exportação de bens e serviços de engenharia pa...,rj,argentina,1554546,2015-03-11,"297015,84","297015,84",...,reembolsável,direta,bndes exim pós-embarque,supplier,ente público,bens e serviços para obras no exterior,liquidada,seguro de crédito / fge,construtora norberto odebrecht,2015
648,construtora norberto odebrecht sa,15.102.288/0001-82,grande,exportação de bens e serviços de engenharia pa...,rj,argentina,1554546,2015-03-11,"561336,51","561336,51",...,reembolsável,direta,bndes exim pós-embarque,supplier,ente público,bens e serviços para obras no exterior,liquidada,seguro de crédito / fge,construtora norberto odebrecht,2015
649,construtora norberto odebrecht sa,15.102.288/0001-82,grande,exportação de bens e serviços de engenharia pa...,rj,argentina,1554546,2015-04-20,"1219965,1","1219965,1",...,reembolsável,direta,bndes exim pós-embarque,supplier,ente público,bens e serviços para obras no exterior,liquidada,seguro de crédito / fge,construtora norberto odebrecht,2015
650,construtora norberto odebrecht sa,15.102.288/0001-82,grande,exportação de bens e serviços de engenharia pa...,rj,argentina,1554546,2015-04-20,"774193,26","774193,26",...,reembolsável,direta,bndes exim pós-embarque,supplier,ente público,bens e serviços para obras no exterior,liquidada,seguro de crédito / fge,construtora norberto odebrecht,2015
